2018/05/31
大学院ゼミ　櫻岡響

# SQLAlchemy使用デモ

# 準備

In [22]:
from functools import reduce
from sklearn.model_selection import train_test_split

from anakin.lang_model.lang_model import LangModel

from anakin.lang_model_file_factory import LangModelFileFactory
from anakin.db.session import database, engine_and_session
from anakin.db.model import Base, Sentence, File, LangModelFile

from anakin.util.word import Word
from anakin.util.estimate import estimate
from anakin.util.key_to_word import KeyToWord
from anakin.util.dp_matching import DPMatching

from env import LOCAL

import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns

In [23]:
db = database(**LOCAL)
ENGINE, Session = engine_and_session(db, echo=False)
Base.prepare(ENGINE, reflect=True)

/Users/hbk/.anyenv/envs/pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.datasets, and will be replaced in the string-lookup table.
  item.__name__
/Users/hbk/.anyenv/envs/pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.migrate_version, and will be replaced in the string-lookup table.
  item.__name__


# 過去に生成した言語モデルファイルインスタンスを呼び出し

In [24]:
session = Session()
lm = session.query(LangModelFile).first()
session.close()

lm

# 冒頭30行を印字

In [25]:
for idx, line in enumerate(lm.contents.decode().split('\n')):
    if idx == 30:
        break
    print(line)

\data\
ngram 1=76934
ngram 2=1550924
ngram 3=1878406

\1-grams:
-1.211105	</s>
-2.441973	<eng>	-1.069783
-1.775336	<num>	-1.903961
-99	<s>	-1.082738
-3.959524	<unk>	-0.4641609
-5.470937	Tシャツ/ティーシャツ	-0.5569668
-7.146104	〆切/シメキリ	-0.1884464
-7.322195	〆張鶴/シメハリヅル	-0.1744659
-5.765893	〇/レイ	-0.6900389
-5.377712	ぁ/ァ	-0.7710243
-6.720135	ぁぁ/ァァ	-0.4356282
-7.146104	ぁぁぁ/ァァァ	-0.581517
-7.623225	ぁぁぁん/ァァァン	-0.1614238
-7.623225	ぁぃさつしてくれてとっても/ァィサツシテクレテトッテモ	-0.188848
-7.623225	ぁい/ァイ	-0.1881081
-7.623225	ぁいあ/ァイア	-0.18832
-7.623225	ぁいいかっ/ァイイカッ	-0.1614238
-7.623225	ぁいいかってかんじでした/ァイイカッテカンジデシタ	-0.1614238
-7.623225	ぁいいのかもしれませんが/ァイイノカモシレマセンガ	-0.1889699
-7.623225	ぁお/ァオ	-0.1888814
-7.623225	ぁきました/ァキマシタ	-0.1614238
-7.623225	ぁこういうこともあるさなのですが/ァコウイウコトモアルサナノデスガ	-0.1889475
-7.623225	ぁこんなものなのかなと/ァコンナモノナノカナト	-0.1614238
-7.623225	ぁこんなもんかなって/ァコンナモンカナッテ	-0.1881081


**言語モデルに使用した文のうち冒頭30文をSentenceクラスのインスタンスとして取得**

In [26]:
session = Session()
lm = session.query(LangModelFile).first()
sentences = lm.sentences[:30]
session.close()

sentences

[<Sentence(id=1, contents='この/コノ ホテル/ホテル で/デ この/コノ 値段/ネダン は/ハ 安い/ヤスイ'),
 <Sentence(id=2, contents='駅/エキ から/カラ 出/デ て/テ すぐ/スグ の/ノ 所/トコロ に/ニ あり/アリ まし/マシ た/タ'),
 <Sentence(id=3, contents='雨/アメ でも/デモ 安心/アンシン です/デス ね/ネ'),
 <Sentence(id=4, contents='乗車/ジョウシャ 券/ケン が/ガ 有効/ユウコウ 利用/リヨウ でき/デキ まし/マシ た/タ'),
 <Sentence(id=5, contents='バス/バス ルーム/ルーム が/ガ 広い/ヒロイ の/ノ が/ガ よかっ/ヨカッ た/タ です/デス'),
 <Sentence(id=6, contents='ラウンジ/ラウンジ で/デ コーヒー/コーヒー を/ヲ 頼ん/タノン だら/ダラ プチケーキ/プチケーキ が/ガ 付い/ツイ て/テ き/キ た/タ'),
 <Sentence(id=7, contents='お/オ 土産/ミヤゲ を/ヲ 買う/カウ とき/トキ に/ニ 便利/ベンリ です/デス ね/ネ'),
 <Sentence(id=8, contents='ウォシュレット/ウォシュレット 付/ヅケ'),
 <Sentence(id=9, contents='食事/ショクジ が/ガ おいしかっ/オイシカッ た/タ です/デス'),
 <Sentence(id=10, contents='<eng> 地下鉄/チカテツ とも/トモ 駅/エキ に/ニ 近く/チカク 時計/トケイ 台/ダイ も/モ そば/ソバ'),
 <Sentence(id=11, contents='ただし/タダシ すすきの/ススキノ に/ニ は/ハ 遠い/トオイ'),
 <Sentence(id=12, contents='歩い/アルイ て/テ <num> 分/フン ほど/ホド で/デ コンビニ/コンビニ 酒/シュ も/モ あり/アリ が/ガ ある/アル の/ノ は/ハ 嬉しい/ウレシイ'),
 <Sentence(id=13, contents='シャンプー/シャンプー ひ/ヒ げ/ゲ そり/ソリ 歯ブラ

In [27]:
sentences[5]

<Sentence(id=6, contents='ラウンジ/ラウンジ で/デ コーヒー/コーヒー を/ヲ 頼ん/タノン だら/ダラ プチケーキ/プチケーキ が/ガ 付い/ツイ て/テ き/キ た/タ')